In [1]:
import numpy as np
import pandas as pd
import string
from sklearn import preprocessing


from tqdm import tqdm
import gensim  ### tester le summarize
from nltk.tokenize import RegexpTokenizer
from textblob import TextBlob
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import re


C:\Users\Franck\Documents\Continuum\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
def read_text_data(filename):
    texts = []
    with open(filename,encoding="utf8") as fi:
        fi.readline()
        for line in fi:
            id, text = line.split("||")
            texts.append(text)
    return texts
train_texts = read_text_data("bases/training_text")
test_texts = read_text_data("bases/test_text")
train_0 = pd.read_csv("bases/training_variants")
test_0 = pd.read_csv("bases/test_variants")

In [4]:
#nltk.download() and download stopwords + WordNet

In [75]:
stop = set(stopwords.words('english'))
exclude = set('!"#$%&\'()*+/:;<=>?@[\\]^_`{|}~') 
exclude_numbers= set('!"#$%&\'()*+/:;<=>?@[\\]^_`{|}~0123456789') 
special_exclude=set('.,')
lemma = WordNetLemmatizer()
def clean(doc,lemmatiz=True):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free_0 = [re.sub(",|\."," ",ch) for ch in stop_free]
    if lemmatiz:
        punc_free_numbers="".join(ch for ch in punc_free_0 if ch not in exclude_numbers)
        normalized = " ".join(lemma.lemmatize(word) for word in punc_free_numbers.split())
        return normalized
    else:
        punc_free = "".join(ch for ch in punc_free_0 if ch not in exclude)
        return punc_free

In [76]:
train_clean = [clean(doc) for doc in train_texts]  
test_clean = [clean(doc) for doc in test_texts]

In [77]:
train_clean_nolemma = [clean(doc,lemmatiz=False) for doc in train_texts]  
test_clean_nolemma = [clean(doc,lemmatiz=False) for doc in test_texts] 

In [79]:
df_train_text=pd.DataFrame(train_clean).reset_index()
df_train_text.columns=["ID","Text"]
df_test_text=pd.DataFrame(test_clean).reset_index()
df_test_text.columns=["ID","Text"]
df_train_text_nolemma=pd.DataFrame(train_clean_nolemma).reset_index()
df_train_text_nolemma.columns=["ID","Text"]
df_test_text_nolemma=pd.DataFrame(test_clean_nolemma).reset_index()
df_test_text_nolemma.columns=["ID","Text"]

In [80]:
train = pd.merge(train_0, df_train_text, how='left', on='ID').fillna('')
train = train.drop(['Class'], axis=1)
test = pd.merge(test_0, df_test_text, how='left', on='ID').fillna('')
train["Gene"]=train["Gene"].str.lower()
train["Variation"]=train["Variation"].str.lower()
test["Gene"]=test["Gene"].str.lower()
test["Variation"]=test["Variation"].str.lower()

In [81]:
train_nolemma = pd.merge(train_0, df_train_text_nolemma, how='left', on='ID').fillna('')
train_nolemma = train_nolemma.drop(['Class'], axis=1)
df_test_text_nolemma=pd.merge(test_0, df_test_text_nolemma, how='left', on='ID').fillna('')
train_nolemma["Gene"]=train_nolemma["Gene"].str.lower()
train_nolemma["Variation"]=train_nolemma["Variation"].str.lower()
test_nolemma["Gene"]=test_nolemma["Gene"].str.lower()
test_nolemma["Variation"]=test_nolemma["Variation"].str.lower()

C:\Users\Franck\Documents\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Franck\Documents\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [82]:
#If the Gene is found in the Text, Gene_Share = 1, same for Variation
df_all = pd.concat((train, test), axis=0, ignore_index=True)
df_all_nolemma=pd.concat((train_nolemma,test_nolemma),axis=0,ignore_index=True)
for df in [df_all,df_all_nolemma]:
    df['Gene_Share'] = df.apply(lambda r: sum([1 for w in r['Gene'].split(' ') if w in r['Text'].split(' ')]), axis=1)
    df['Variation_Share'] = df.apply(lambda r: sum([1 for w in r['Variation'].split(' ') if w in r['Text'].split(' ')]), axis=1)
    for c in df.columns:
        if df[c].dtype == 'object':
            if c == "Variation": 
                df[c+'_len'] = df[c].map(lambda x: len(str(x)))
                df[c+'_words'] = df[c].map(lambda x: len(str(x).split(' ')))
            if c == "Gene": 
                df[c+'_len'] = df[c].map(lambda x: len(str(x)))
            if c=='Text': 
                df[c+'_len'] = df[c].map(lambda x: len(str(x)))
                df[c+'_words'] = df[c].map(lambda x: len(str(x).split(' '))) 

In [83]:
#df_all=pd.get_dummies(df_all,columns=["Gene","Variation"])     The one-encoding here is too expensive for not a lot of infos.

In [84]:
gen_var_lst = sorted(list(train.Gene.unique()) + list(train.Variation.unique()))
gen_var_lst_nolemma = sorted(list(train_nolemma.Gene.unique()) + list(train_nolemma.Variation.unique()))
gen_var_lst = [x for x in gen_var_lst if len(x.split(' '))==1] #Suppress weird documented mutation
gen_var_lst_nolemma = [x for x in gen_var_lst_nolemma if len(x.split(' '))==1] #Suppress weird documented mutation

#for the 3091 gen and unique variation we count how many times it appears in the text
for gen_var_lst_itm in gen_var_lst:
    df['GV_'+str(gen_var_lst_itm)] = df_all['Text'].map(lambda x: str(x).count(str(gen_var_lst_itm)))
for gen_var_lst_itm in gen_var_lst_nolemma:
    df['GV_'+str(gen_var_lst_itm)] = df_all_nolemma['Text'].map(lambda x: str(x).count(str(gen_var_lst_itm)))
figure_counter=["fig","figure"]

In [85]:
for df in [df_all,df_all_nolemma]:
    for fig in figure_counter: #On pondère par le nombre de mots, si l'on pondérait par la
    #taille totale, on aurait beaucoup de pertes. Alors que là on a un indicateur
    #qui a tendance à converger vers la vraie information. Effet vocabulaire très présent
    #au début, insensible quand le texte est long.
        df["Figure_counter"]=df["Text"].map(lambda x : str(x).count(fig))/df["Text_words"]

In [86]:
df_all["Text"]=df_all["Text"].replace("fig|figure"," ")
df_all_nolemma["Text"]=df_all_nolemma["Text"].replace("fig|figure"," ")

In [87]:
df_all["Text"]=[re.sub("fig|figure","",doc) for doc in df_all["Text"]]
df_all_nolemma["Text"]=[re.sub("fig|figure","",doc) for doc in df_all_nolemma["Text"]]

In [88]:
train = df_all.iloc[:len(train)]
test = df_all.iloc[len(train):]
train_nolemma = df_all_nolemma.iloc[:len(train_nolemma)]
test_nolemma = df_all_nolemma.iloc[len(train_nolemma):]

In [89]:
train.to_csv("bases/working_train.csv",index=False,encoding="utf8")
test.to_csv("bases/working_test.csv",index=False,encoding="utf8")
train_nolemma.to_csv("bases/working_train_nolemma.csv",index=False,encoding="utf8")
test_nolemma.to_csv("bases/working_test_nolemma.csv",index=False,encoding="utf8")

In [92]:
train_nolemma["Text"][1]

'abstract background non-small cell lung cancer nsclc heterogeneous group disorders number genetic proteomic alterations  c-cbl e3 ubiquitin ligase adaptor molecule important normal homeostasis cancer  determined genetic variations c-cbl  relationship receptor tyrosine kinases egfr met  functionality nsclc  methods findings using archival formalin-fixed paraffin embedded ffpe extracted genomic dna  show c-cbl mutations occur somatic fashion lung cancers  c-cbl mutations mutually exclusive met egfr mutations however independent p53 kras mutations  normaltumor pairwise analysis  significant loss heterozygosity loh c-cbl locus 22  n  837 none samples revealed mutation remaining copy c-cbl  c-cbl loh also positively correlated egfr met mutations observed samples  using select c-cbl somatic mutations s80nh94y  q249e w802 obtained caucasian  taiwanese african-american samples  respectively transfected nsclc cell lines  increased cell viability cell motility  conclusions taking overall mutati